# HTML scraping and GraphQL

## Goal

- Scrape Wikipedia HTML (ex. Cities in Japan) and generate pandas DataFrame
- Insert data of DataFrame into database via GraphQL

## Preparation

Install libraries in terminal.

```sh
pip install graphqlclient numpy pandas lxml
```

In notebook,

```py
!pip install graphqlclient numpy pandas lxml
```

In [1]:
import numpy as np
import pandas as pd

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# HTML scraping

In [2]:
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_cities_in_Japan"
dfs = pd.read_html(WIKI_URL, header=0) # set header with row #0

In [3]:
# get 3rd table
df = dfs[2]

In [4]:
# add country column
df['Country']='Japan'
df = df.drop('Homepage', axis=1)

In [5]:
# check
df[df['City (Special Ward)']=='Yokohama']

,City (Special Ward),Japanese,Prefecture,Population,Area,Density,Founded,Country
354,Yokohama,横浜市,Kanagawa,3697894,437.38,8500.0,1889-04-01,Japan


In [39]:
json=df.to_json(orient='table',path_or_buf='./japancities.json')

In [38]:
json

# GraphQL

## Preparation

In [31]:
from graphqlclient import GraphQLClient
import json
0
client = GraphQLClient('http://10.0.2.2:4021/')

def execMutationCreateLocation(address, country):
    location = {
        'address': address,
        'country': country
    }
    result = client.execute('''
    mutation CreateLocationMutation($address: String!, $country: String!) {
    createLocation(data: {
        address: $address
        country: $country
        test: {
          connect: {
            test: "new"
          }
        }
      }) {
        address
        country
      }
    }
    ''', variables=json.dumps(location))

def iterateExecMutation(df):
    for _, row in df.iterrows():
        execMutationCreateLocation(address=row['City (Special Ward)'], country=row['Country'])


## GraphQL Execution (a. Single thread input)

In [32]:
%%time
iterateExecMutation(df)

b'{"errors":[{"message":"Field \\"test\\" is not defined by type TestWhereUniqueInput.","locations":[{"line":8,"column":13}]}]}'



HTTPError: HTTP Error 400: Bad Request

## GraphQL Execution (b. Parallel input)

In [14]:
# split dataframe into 4 slices
dflist = []
SLICE_NUM = 4
for i in range(SLICE_NUM):
    dflist.append(df.iloc[i::SLICE_NUM])

In [34]:
%%time
from multiprocessing import Pool
p = Pool(4)
p.map(iterateExecMutation, dflist)

CPU times: user 140 ms, sys: 160 ms, total: 300 ms
Wall time: 12.8 s


# Appendix

## Processing time (single thread input)

creation rows #: 815

### Batch 

CPU times: user 1.64 s, sys: 816 ms, total: 2.46 s
Wall time: 43.3 s

CPU times: user 1.64 s, sys: 832 ms, total: 2.48 s
Wall time: 34.5 s

CPU times: user 1.91 s, sys: 858 ms, total: 2.77 s
Wall time: 38.5 s

### Single

CPU times: user 1.28 s, sys: 555 ms, total: 1.84 s
Wall time: 27.6 s

CPU times: user 1.45 s, sys: 681 ms, total: 2.13 s
Wall time: 30.2 s

CPU times: user 1.63 s, sys: 784 ms, total: 2.42 s
Wall time: 29.4 s

## Processing time (parallel input)

creation rows #: 815

### Batch 

CPU times: user 77 ms, sys: 297 ms, total: 374 ms
Wall time: 20.4 s

CPU times: user 106 ms, sys: 219 ms, total: 325 ms
Wall time: 16.9 s

CPU times: user 142 ms, sys: 102 ms, total: 245 ms
Wall time: 15.4 s

CPU times: user 150 ms, sys: 562 ms, total: 712 ms
Wall time: 18.8 s

### Single

CPU times: user 258 ms, sys: 182 ms, total: 440 ms
Wall time: 20.3 s

CPU times: user 242 ms, sys: 425 ms, total: 667 ms
Wall time: 21.9 s

CPU times: user 302 ms, sys: 427 ms, total: 730 ms
Wall time: 21.7 s


## Memo

In [20]:
from graphqlclient import GraphQLClient

client = GraphQLClient('http://10.0.2.2:4000')

result = client.execute('''
{
  users {
    ...namename
  }
}

fragment namename on User {
  name
  id
  __typename
}
''')

print(result)

{"data":{"users":[{"name":"user1","id":"cjkrq5vtv0048073925e1yf6s","__typename":"User"},{"name":"user2","id":"cjkrq6319004c0739o64pqjb4","__typename":"User"},{"name":"user3","id":"cjkrq6dda004g0739milhdgsb","__typename":"User"},{"name":"user4","id":"cjkyiszsx000o07394qoto31s","__typename":"User"}]}}


In [38]:
from graphqlclient import GraphQLClient

client = GraphQLClient('http://10.0.2.2:4000')

result = client.execute('''
mutation ExampleMutation($name: String!, $address: String!) {
  createUser(data: {
    name: $name
    address: $address
  }) {
    id
    name
    address
  }
}
''', variables='''
{
  "name": "user4",
  "address": "Yokohama"
}
''')

print(result)

{"data":{"createUser":{"id":"cjkyiszsx000o07394qoto31s","name":"user4","address":"Yokohama"}}}
